In [ ]:
# This code was primarily written by Otavio Alves (oalves@umich.edu)
# with changes by Jay Wadekar

# Needs nbodykit installed (https://nbodykit.readthedocs.io/en/latest/getting-started/install.html)
# and files for the survey random catalog

In [163]:
import numpy as np
from numpy import exp, log, log10, cos, sin, pi, cosh, sinh , sqrt, amin, amax, mean, dot, power, conj
import dask.array as da
from matplotlib import pyplot as plot
import matplotlib.pyplot as plt
import itertools as itt
import fitsio
from nbodykit.source.catalog import FITSCatalog, CSVCatalog
from nbodykit.lab import *
from scipy import fft

In [164]:
cosmo = cosmology.Cosmology(h=0.7).match(Omega0_m=0.31)
Nmesh = 48 # FFT size
BoxSize = 3750. #Box size, should encompass all the galaxies

In [165]:
# Download the BOSS random catalog from https://data.sdss.org/sas/dr12/boss/lss/
# RA, Dec, z, w_fkp, nbar are given by columns 0,1,2,3,4 in the fits files
# Reference: https://nbodykit.readthedocs.io/en/latest/catalogs/reading.html?highlight=fits#FITS-Data

randoms = FITSCatalog('/data/jayw/NYU/Covariance/Data/BOSS/catalogs/random0_DR12v5_CMASSLOWZTOT_North.fits')
randoms = randoms[(randoms['Z'] > 0.5) * (randoms['Z'] < 0.75)]

In [166]:
randoms['W12'] = randoms['WEIGHT_FKP']**2 
randoms['W22'] = (randoms['WEIGHT_FKP']**2) * randoms['NZ']

In [167]:
# Calculating Iij

#I22 is calculated as an example here (note difference in the nbar power as nbar also enters in the sum)
I22 = da.sum(randoms['NZ'] * randoms['WEIGHT_FKP']**2)
I22.compute()

437.18336496809707

In [168]:
randoms['OriginalPosition'] = transform.SkyToCartesian(
    randoms['RA'], randoms['DEC'], randoms['Z'], degrees=True, cosmo=cosmo)

x = randoms['OriginalPosition'].T

# Shifting the points such that the survey center is in the center of the box
randoms['Position'] = randoms['OriginalPosition'] + da.array(3*[BoxSize/2])

In [169]:
# Types of fourth-order FFTs that will be generated below

w='W22'
label=[]
for (i,i_label),(j,j_label),(k,k_label),(l,l_label) in itt.combinations_with_replacement(enumerate(['x', 'y', 'z']), r=4):
        label.append(w + i_label + j_label + k_label + l_label)

print(', '.join(label))

W22xxxx, W22xxxy, W22xxxz, W22xxyy, W22xxyz, W22xxzz, W22xyyy, W22xyyz, W22xyzz, W22xzzz, W22yyyy, W22yyyz, W22yyzz, W22yzzz, W22zzzz


In [172]:
num_ffts = lambda n: int((n+1)*(n+2)/2)

export=np.zeros((2*(1+num_ffts(2)+num_ffts(4)),Nmesh,Nmesh,Nmesh),dtype='complex128')

In [173]:
# Warning: computing this cell takes a bit of time

ind=0

for w in ['W22', 'W12']:
    print(f'Computing FFTs of {w}')
    
    print('Computing 0th order FFTs')
    Wij = np.fft.fftn(randoms.to_mesh(Nmesh=Nmesh, BoxSize=BoxSize, value=w, resampler='tsc', interlaced=True, compensated=True).paint())
    Wij *= (da.sum(randoms[w]).compute())/np.real(Wij[0,0,0]) #Fixing normalization, e.g., zero mode should be I22 for 'W22'
    export[ind]=Wij; ind+=1
    
    print('Computing 2nd order FFTs')
    for (i,i_label),(j,j_label) in itt.combinations_with_replacement(enumerate(['x', 'y', 'z']), r=2):
        label = w + i_label + j_label
        randoms[label] = randoms[w] * x[i]*x[j] /(x[0]**2 + x[1]**2 + x[2]**2)
        Wij = np.fft.fftn(randoms.to_mesh(Nmesh=Nmesh, BoxSize=BoxSize, value=label, resampler='tsc', interlaced=True, compensated=True).paint())
        Wij *= (da.sum(randoms[label]).compute())/np.real(Wij[0,0,0])
        export[ind]=Wij; ind+=1

    print('Computing 4th order FFTs')
    for (i,i_label),(j,j_label),(k,k_label),(l,l_label) in itt.combinations_with_replacement(enumerate(['x', 'y', 'z']), r=4):
        label = w + i_label + j_label + k_label + l_label
        randoms[label] = randoms[w] * x[i]*x[j]*x[k]*x[l] /(x[0]**2 + x[1]**2 + x[2]**2)**2
        Wij = np.fft.fftn(randoms.to_mesh(Nmesh=Nmesh, BoxSize=BoxSize, value=label, resampler='tsc', interlaced=True, compensated=True).paint())
        Wij *= (da.sum(randoms[label]).compute())/np.real(Wij[0,0,0])
        export[ind]=Wij; ind+=1
        

Computing FFTs of W22
Computing 0th order FFTs
Computing 2nd order FFTs
Computing 4th order FFTs
Computing FFTs of W12
Computing 0th order FFTs
Computing 2nd order FFTs
Computing 4th order FFTs


In [174]:
export.shape

(44, 48, 48, 48)

In [175]:
np.save('/data/jayw/NYU/Covariance/Data/FFTWinFun_HighZ_NGC.npy',export)